In [1]:
import os
from openai import OpenAI
client = OpenAI()

client.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
assistant = client.beta.assistants.create(
    name="국어 선생님",
    instructions="당신은 친절한 국어선생님입니다. 사용자가 질문하는 국어문제에 대답하기 위해 File Search를 실행하세요.",
    tools=[{"type": "file_search"}],
    model="gpt-4o-mini"
)
assistant

Assistant(id='asst_seHsd9Tl4Ixf9fTQen02AKFE', created_at=1737534745, description=None, instructions='당신은 친절한 국어선생님입니다. 사용자가 질문하는 국어문제에 대답하기 위해 File Search를 실행하세요.', metadata={}, model='gpt-4o-mini', name='국어 선생님', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

In [3]:
file = client.files.create(
  file=open("소나기.pdf", "rb"),
  purpose='assistants'
)
file

FileObject(id='file-9tUFNB7jCLBfCH6N3jLEPA', bytes=123544, created_at=1737534789, filename='소나기.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [4]:
thread = client.beta.threads.create()
thread

Thread(id='thread_JVDdRapEseHJcwgkWYNjOrNi', created_at=1737534804, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="소년과 소녀는 어디에서 처음 만났나?",
    attachments=[
        { "file_id": file.id, "tools": [{"type": "file_search"}] }
    ]
)

In [6]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[Message(id='msg_kYFuWzG1ZywMbC6X7ATnPrWR', assistant_id=None, attachments=[Attachment(file_id='file-9tUFNB7jCLBfCH6N3jLEPA', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='소년과 소녀는 어디에서 처음 만났나?'), type='text')], created_at=1737534837, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_JVDdRapEseHJcwgkWYNjOrNi')]


In [7]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="주어진 질문에 대해 답을 줘. 답을 모르겠으면 모르겠다고 답해야 하고 답을 지어내면 안돼."
)

In [8]:
import time

while run.status != "completed"  :
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    time.sleep(0.1)

In [9]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [10]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[Message(id='msg_9SoQa7HLlLaAEXdB0LWvTVQW', assistant_id='asst_seHsd9Tl4Ixf9fTQen02AKFE', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=81, file_citation=FileCitation(file_id='file-9tUFNB7jCLBfCH6N3jLEPA'), start_index=72, text='【4:0†소나기】', type='file_citation')], value='소년과 소녀는 개울가에서 처음 만났습니다. 소녀는 개울에서 물장난을 하고 있었고, 소년은 그 모습을 보며 개울둑에 앉아 있었습니다【4:0†소나기】.'), type='text')], created_at=1737534869, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_mIBc0r5Kok71eC4YtxldssmO', status=None, thread_id='thread_JVDdRapEseHJcwgkWYNjOrNi'), Message(id='msg_kYFuWzG1ZywMbC6X7ATnPrWR', assistant_id=None, attachments=[Attachment(file_id='file-9tUFNB7jCLBfCH6N3jLEPA', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='소년과 소녀는 어디에서 처음 만났나?'), type='text')], created_at=173

In [11]:
print(thread_messages.data[0].role)
print(thread_messages.data[0].content[0].text.value)

assistant
소년과 소녀는 개울가에서 처음 만났습니다. 소녀는 개울에서 물장난을 하고 있었고, 소년은 그 모습을 보며 개울둑에 앉아 있었습니다【4:0†소나기】.
